# Weights vs Attention

I'm curious _which_ I should choose the attention of the version of the filter normalized over all the pixels.

In [25]:
import torch 
import torch.nn.functional as F
import numpy as np

from scipy.optimize import linear_sum_assignment


In [21]:
bs = 1
max_n_rings = 2
k_slots = 3
nPixels=2

In [159]:
att = torch.Tensor([[.9,.9,.9,.9],
                     [.1,0,0,0],
                     [0,.1,.1,.1]]).unsqueeze(0)

In [160]:
att.shape

torch.Size([1, 3, 4])

In [106]:
ws = att / att.sum(dim=2,keepdim=True)
ws

tensor([[[0.2500, 0.2500, 0.2500, 0.2500],
         [1.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.3333, 0.3333, 0.3333]]])

In [107]:
ws

tensor([[[0.2500, 0.2500, 0.2500, 0.2500],
         [1.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.3333, 0.3333, 0.3333]]])

In [108]:
mask

tensor([[[1., 0., 0., 0.],
         [0., 0., 0., 1.]]])

In [109]:
mask = torch.Tensor(
    [[1,0,0,0],[0,0,0,1]]
).unsqueeze(0)

In [110]:
mask_ext

tensor([[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.]])

In [111]:
att_ext

tensor([[0.2500, 0.2500, 0.2500, 0.2500],
        [1.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3333, 0.3333, 0.3333],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [1.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3333, 0.3333, 0.3333]])

In [112]:
pairwise_cost

tensor([[[ 0.5623, 50.0000, 25.3041],
         [ 0.5623,  0.0000,  0.4774]]])

In [156]:
att.shape

torch.Size([1, 3, 4])

In [162]:
att_ext  = torch.tile(att.unsqueeze(2),  dims=(1,1,max_n_rings,1)) #.reshape(bs * k_slots * max_n_rings , nPixels**2)
mask_ext = torch.tile(mask,dims=(1,k_slots,1,1)) #.reshape(bs * k_slots * max_n_rings , nPixels**2)

print('att_ext',att_ext.shape)
print('mask_ext',mask_ext.shape)

pairwise_cost = F.binary_cross_entropy(att_ext,mask_ext,reduction='none').mean(axis=-1)
# pairwise_cost = pairwise_cost.reshape(bs, k_slots,max_n_rings)

indices = list(map(linear_sum_assignment, pairwise_cost.cpu()))
indices = torch.LongTensor(indices)

transposed_indices = torch.permute(indices,(0,2,1))

# return indices

att_ext torch.Size([1, 3, 2, 4])
mask_ext torch.Size([1, 3, 2, 4])


In [116]:
indices.shape

torch.Size([1, 2, 2])

In [163]:
pairwise_cost[0] #.shape

tensor([[ 1.7533,  1.7533],
        [ 0.5756, 25.0263],
        [25.0790,  0.6283]])

In [164]:
for pi, (ri,ci) in zip(pairwise_cost,indices):
    break

pi[ri,ci]

tensor([0.5756, 0.6283])

In [93]:
ri

tensor([1, 2])

In [94]:
ci

tensor([1, 0])

In [82]:
mask.shape

torch.Size([2, 4])

In [79]:
bis=torch.arange(1)

slots_sorted = torch.cat([ws[bis,indices[:,0,ri]].unsqueeze(1) for ri in range(max_n_rings)],dim=1)

# flat_mask = mask.reshape(-1,max_n_rings, nPixels*nPixels)
rings_sorted = torch.cat([mask[bis,indices[:,1,ri]].unsqueeze(1) for ri in range(max_n_rings)],dim=1)

# Calculate the loss
# loss = F.binary_cross_entropy(slots_sorted,rings_sorted,reduction='none').sum(axis=1).mean()


In [80]:
slots_sorted

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.3333, 0.3333, 0.3333]]])

In [81]:
rings_sorted

tensor([[0., 1.]])

In [78]:
F.binary_cross_entropy(slots_sorted,rings_sorted,reduction='none').sum(axis=1).mean()

tensor(75.3041)

**Just try to understand the string manipulation!**

In [100]:
att.shape

torch.Size([1, 3, 4])

In [ ]:
# att = torch.Tensor([[10.1, 10.2, 10.3, 10.4],
#                     [20.1, 20.2, 20.3, 20.4],
#                     # [30.1, 30.2, 30.3, 30.4],
#                    ])
# mask = 

In [135]:
pred = 10* (torch.arange(k_slots)+1).reshape(1,-1,1)

trgt = (torch.arange(max_n_rings)+1).reshape(1,1,-1)


In [136]:
pred

tensor([[[10],
         [20],
         [30]]])

In [137]:
trgt

tensor([[[1, 2]]])

In [138]:
pred_tile = torch.tile(pred,dims=(1,1,max_n_rings))

In [139]:
trgt_tile = torch.tile(trgt,dims=(1,k_slots,1))

In [140]:
pred_tile+trgt_tile

tensor([[[11, 12],
         [21, 22],
         [31, 32]]])

In [143]:
pred_flat = pred_tile.reshape(1,k_slots*max_n_rings)
trgt_flat = trgt_tile.reshape(1,k_slots*max_n_rings)

In [145]:
(pred_flat+trgt_flat).reshape((1,k_slots,max_n_rings))

tensor([[[11, 12],
         [21, 22],
         [31, 32]]])

**OK, I made a dummy example where reshaping _worked_**

So... I'm not sure what wasn't working... was it not working when I have other dimensions involved?

In [149]:
pred = 10* (torch.arange(k_slots)+1).reshape(1,-1,1,1) + 0.001*torch.randn(1,k_slots,1,2)

trgt = (torch.arange(max_n_rings)+1).reshape(1,1,-1,1) + 0.001*torch.randn(1,1,max_n_rings,2)


In [150]:
pred

tensor([[[[ 9.9999,  9.9991]],

         [[19.9988, 19.9977]],

         [[29.9994, 30.0000]]]])

In [152]:
trgt

tensor([[[[0.9995, 0.9990],
          [1.9982, 1.9999]]]])

In [ ]:
pred_flat = pred_tile.reshape(1,k_slots*max_n_rings)
trgt_flat = trgt_tile.reshape(1,k_slots*max_n_rings)